In [ ]:
# Model code starts here
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Correct import
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

# Create an ImageDataGenerator to rescale pixel values
datagen = ImageDataGenerator(rescale=1.0/255.0)  

# Load the images from folders
train_data = datagen.flow_from_directory(
    'dataset/train',  # REPLACE THIS FOR YOUR DIRECTORY
    target_size=(64, 64),  # Resize images to 64x64 pixels
    batch_size=32,         # Load images in batches of 32
    class_mode='binary'    # Since we have two types (fake vs real) use binary mode
)

In [2]:
# Build model
model = Sequential()

# Add an explicit Input layer
model.add(Input(shape=(64, 64, 3)))  # Input layer specifying the image size and 3 channels (RGB)

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu'))  # No need for input_shape here
model.add(MaxPooling2D(pool_size=(2, 2)))  # Max pooling to reduce size

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps into a 1D vector
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))

# Output layer (binary classification: fake vs real)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
#model.summary()

In [ ]:
# Train the model
model.fit(train_data, epochs=5)  

In [ ]:
# Test images here
def predict_images_from_directory(directory):
    results = []

    if not os.path.exists(directory):
        print(f"Directory {directory} does not exist.")
        return []

    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=(64, 64))  # Use load_img from keras
            img = img_to_array(img)  # Convert image to array
            img = np.expand_dims(img, axis=0)
            img = img / 255.0  # Normalize  image
            
            prediction = model.predict(img)

            if prediction[0][0] >= 0.5:
                results.append((filename, 'Real'))
            else:
                results.append((filename, 'Fake'))

    return results
    
# Path to your directory 
new_image_directory = 'dataset/t'

# Get predictions
predictions = predict_images_from_directory(new_image_directory)

for filename, label in predictions:
    print(f"{filename}: {label}")


In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score

# Define the directory of test images (organized into subfolders 'Real' and 'Fake')
test_directory = 'dataset/test'  # Replace with your test data directory

# Create an ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(rescale=1./255)

# Create a generator for the test set
test_generator = datagen.flow_from_directory(
    test_directory,
    target_size=(64, 64),  # Resize images to match your model's expected input
    batch_size=32,  # You can adjust this value based on your system's memory
    class_mode='binary',  # Binary classification (Real or Fake)
    shuffle=False  # Do not shuffle, as we need to match predictions with true labels
)

# Get the total number of samples
num_samples = test_generator.samples

# Predict using the model on the generator
y_pred_probs = model.predict(test_generator, steps=int(np.ceil(num_samples / test_generator.batch_size)))
y_pred = (y_pred_probs >= 0.5).astype(int).flatten()  # Threshold for binary classification

# Get true labels from the generator
y_true = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification Report (includes Precision, Recall, and F1-Score)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Fake', 'Real']))

# Precision, Recall, F1-score (you can calculate them separately too)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Plot confusion matrix for better visualization
plt.figure(figsize=(4, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()
